In [1]:
import nltk
from nltk import word_tokenize, pos_tag, pos_tag_sents
#nltk.download('wordnet')
#nltk.download('vader_lexicon')
import os.path
import re
import pandas as pd 
import numpy as np 
from nltk.corpus import wordnet as wn
from textblob import TextBlob
from afinn import Afinn
import matplotlib.pyplot as plt
nltk.download('vader_lexicon')



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/alinazhiltsova/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
os. chdir("/Users/alinazhiltsova/Desktop/NCI/Thesis/Code")

In [3]:
#loading etymological wordnet for extracting English words with French origin
etym = pd.read_csv("etymwn.tsv",sep = '\t')
#etym.head()


In [3]:
#loading etymological wordnet for extracting English words with French origin
etym = pd.read_csv("etymwn.tsv",sep = '\t')
#etym.head()

#loading English reddit posts written by French speakers
french = pd.read_csv("reddit_french.csv")

#filtering the EW to get English words
is_eng = etym[etym['eng: Penobscot'].str.contains("eng:")]
#is_eng.shape
#(991181, 3)

#filtering the English words to keep only those that have French origin
fr_eng = is_eng[is_eng['aaq: Pawanobskewi'].str.contains("fra:")]
#fr_eng.shape
#(2904, 3)

#renaming the columns for convenience
fr_eng.rename({'aaq: Pawanobskewi': 'Origin',
                                          'rel:etymological_origin_of': 'Relation', 
                                          'eng: Penobscot':'Word'}, axis=1, inplace=True)
                
#filtering the df to keep only 'etymological origin of' type of relationship between the words
orig_fr_eng = fr_eng[fr_eng['Relation'].str.contains("rel:etymological_origin_of")]
#2633 rows × 3 columns

#cleaning the rows to remove the first 5 symbols, because they are the same in every row
orig_fr_eng['Word'] = orig_fr_eng['Word'].str[5:]
orig_fr_eng['Origin'] = orig_fr_eng['Origin'].str[5:]
#a warning comes up, but everything works fine - no action required

#keeping only words starting with a lowercase letter
#words starting with a capital letter are likely to be names
#doing this we also remove suffixes
orig_fr_eng = orig_fr_eng[orig_fr_eng.Origin.apply(lambda x: x[0].islower())]
#orig_fr_eng
#2514 rows × 3 columns

french.loc[:,'post']= french.post.apply(lambda x : str.lower(x))
french.loc[:,'post'] = french.post.apply(lambda x : " ".join(re.findall('[\w]+',x)))
#https://stackoverflow.com/questions/47947438/preprocessing-string-data-in-pandas-dataframe

wordListFr = orig_fr_eng["Word"].tolist()
wordSetFr = set(wordListFr)

french['lst'] = french['post'].tolist()
lst = french['lst'].tolist()
lst_words = [set(line.split(' ')) for line in lst]


lst_matches = [list(wordSetFr.intersection(line_wordset)) for line_wordset in lst_words]  # list comprehension
fr = [l for l in list(zip(lst, lst_matches)) if l[1]]

mtch = [match[1] for match in fr] 
#len(mtch)
#14136

matchlist = sum(mtch, [])
#len(matchlist)
#60059

matchlist_unique = list(set(matchlist))
#len(matchlist_unique)
#910

matchlist_unique.sort()

#to check if the word is in the list
#matchlist_unique.count('word')

n_synsets = []
for x in matchlist_unique:
    n_synsets.append(wn.synsets((x), pos='n')) 
    

sns = pd.DataFrame(
    {'word': matchlist_unique,
     'synsets': n_synsets
    })

sns = sns[sns['synsets'].map(lambda d: len(d)) > 0]        

sns1 = (sns.apply(lambda x: x.str[0]))
sns['synset1'] = sns1 ['synsets']

lemmas = []
for x in sns.synset1:
    lemmas.append((x).lemma_names())
sns['lemmas'] = lemmas    

#remove words that do not have synonyms
sns = sns[sns['lemmas'].map(lambda d: len(d)) > 1]

sns['1st_synonym'] = sns.lemmas.map(lambda x: x[0]) 
sns['2nd_synonym'] = sns.lemmas.map(lambda x: x[1]) 
sns['first_distinct_pair'] = (sns['word'] + ' '+ sns['1st_synonym']).where(sns['word'] != sns['1st_synonym'], 
                                                   sns['word'] + ' ' + sns['2nd_synonym'])

nouns = pd.DataFrame(columns=['words','type'])
nouns['words']= sns['first_distinct_pair']
nouns['type'] = 'object'

/Users/alinazhiltsova/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Users/alinazhiltsova/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/alinazhiltsova/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata

In [58]:
nouns = pd.DataFrame(columns=['words','type'])
nouns['words']= sns['first_distinct_pair']
nouns['type'] = 'object'

In [59]:
nouns.shape

(440, 2)

In [61]:
french = pd.read_csv("reddit_french.csv")


In [62]:
french

,user,subreddit,post
0,0j0s,AskEurope,Most of my programs on the Atari were in Engli...
1,0j0s,AskEurope,Merci pour la conversion en km !
2,0kZ,europe,"The ring of fiiiire , the ring of fiiire !"
3,0kZ,europe,Why would he lift heavy steel bars with steel ...
4,0kZ,europe,I thought we were friends and that the enemies...
5,0kZ,europe,I would watch it happily .
6,0kZ,europe,You\\ 're talking to a guy that username is \ ...
7,0kZ,europe,"Given your username , I sincerely and unoffici..."
8,0kZ,europe,This is pretty much the what one of the main d...
9,0kZ,europe,GILF ?


In [53]:
nouns[nouns.isin(['soporific hypnotic'])].dropna(how='all')

,words,type
824,soporific hypnotic,NaN


In [54]:
nouns = nouns[nouns.words != 'adept ace']
nouns = nouns[nouns.words != 'aeronautics astronautics']
nouns = nouns[nouns.words != 'venal corruptible']
nouns = nouns[nouns.words != 'equitable just']
nouns = nouns[nouns.words != 'valley vale']
nouns = nouns[nouns.words != 'anatomy general_anatomy']
nouns = nouns[nouns.words != 'annuity rente']
nouns = nouns[nouns.words != 'annex annexe']
nouns = nouns[nouns.words != 'annelid annelid_worm']
nouns = nouns[nouns.words != 'antibiotic antibiotic_drug']
nouns = nouns[nouns.words != 'anomie anomy']
nouns = nouns[nouns.words != 'apanage appanage']
nouns = nouns[nouns.words != 'appanage apanage']
nouns = nouns[nouns.words != 'apologetic excusatory']
nouns = nouns[nouns.words != 'apprise advise']
nouns = nouns[nouns.words != 'admire look_up_to']
nouns = nouns[nouns.words != 'babiroussa babirusa']
nouns = nouns[nouns.words != 'barcarolle barcarole']
nouns = nouns[nouns.words != 'are ar']
nouns = nouns[nouns.words != 'automatize automatise']
nouns = nouns[nouns.words != 'arctic Arctic']
nouns = nouns[nouns.words != 'aye-aye Daubentonia_madagascariensis']
nouns = nouns[nouns.words != 'aerobic aerophilic']
nouns = nouns[nouns.words != 'banal commonplace']
nouns = nouns[nouns.words != 'bandelet annulet']
nouns = nouns[nouns.words != 'bimetallic bimetallistic']
nouns = nouns[nouns.words != 'bonduc Kentucky_coffee_tree']
nouns = nouns[nouns.words != 'aioli aioli_sauce']
nouns = nouns[nouns.words != 'basketball basketball_game']
nouns = nouns[nouns.words != 'tuff tufa']
nouns = nouns[nouns.words != 'tuner piano_tuner']
nouns = nouns[nouns.words != 'tutu Tutu']
nouns = nouns[nouns.words != 'telecommunication telecom']
nouns = nouns[nouns.words != 'unique alone']
nouns = nouns[nouns.words != 'equip fit']
nouns = nouns[nouns.words != 'caffeine caffein']
nouns = nouns[nouns.words != 'brilliant superb']
nouns = nouns[nouns.words != 'carrousel carousel']
nouns = nouns[nouns.words != 'capricious freakish']
nouns = nouns[nouns.words != 'chanterelle chantarelle']
nouns = nouns[nouns.words != 'subpoena subpoena_ad_testificandum']
nouns = nouns[nouns.words != 'svelte polished']
nouns = nouns[nouns.words != 'symptomatic diagnostic']
nouns = nouns[nouns.words != 'syndicate crime_syndicat']
nouns = nouns[nouns.words != 'tableau tableau_vivant']
nouns = nouns[nouns.words != 'tarot tarot_card']
nouns = nouns[nouns.words != 'serval Felis_serval']
nouns = nouns[nouns.words != 'sirup syrup']
nouns = nouns[nouns.words != 'slang slang_expression']
nouns = nouns[nouns.words != 'somatic bodily']
nouns = nouns[nouns.words != 'sourdine sordino']
nouns = nouns[nouns.words != 'spencer Spencer']
nouns = nouns[nouns.words != 'sponsoring sponsor']
nouns = nouns[nouns.words != 'renaissance Renaissance']
nouns = nouns[nouns.words != 'repent atone']
nouns = nouns[nouns.words != 'revivify animate']
nouns = nouns[nouns.words != 'riant laughing']
nouns = nouns[nouns.words != 'risible amusing']
nouns = nouns[nouns.words != 'ricochet carom']
nouns = nouns[nouns.words != 'rose rosebush']
nouns = nouns[nouns.words != 'reactionary ultraconservative']
nouns = nouns[nouns.words != 'rebarbative repellent']
nouns = nouns[nouns.words != 'identify place']
nouns = nouns[nouns.words != 'idealise idealize']
nouns = nouns[nouns.words != 'ignore disregard']
nouns = nouns[nouns.words != 'implore beg']
nouns = nouns[nouns.words != 'impute ascribe']
nouns = nouns[nouns.words != 'inaugural inaugural_address']
nouns = nouns[nouns.words != 'inaugurate kick_off']
nouns = nouns[nouns.words != 'inept awkward']
nouns = nouns[nouns.words != 'inestimable incomputable']
nouns = nouns[nouns.words != 'insouciant casual']
nouns = nouns[nouns.words != 'jovial gay']
nouns = nouns[nouns.words != 'kilogram kg']
nouns = nouns[nouns.words != 'kilometre kilometer']
nouns = nouns[nouns.words != 'la lanthanum']
nouns = nouns[nouns.words != 'lateen lateen_sail']
nouns = nouns[nouns.words != 'galvanic voltaic']
nouns = nouns[nouns.words != 'gauche graceless']
nouns = nouns[nouns.words != 'genteel civilized']
nouns = nouns[nouns.words != 'gerbil gerbille']
nouns = nouns[nouns.words != 'gerbille gerbil']
nouns = nouns[nouns.words != 'girandole girandola']
nouns = nouns[nouns.words != 'gelatin gelatine']
nouns = nouns[nouns.words != 'gratify satisfy']
nouns = nouns[nouns.words != 'hydrogen H']
nouns = nouns[nouns.words != 'hypnagogic soporific']
nouns = nouns[nouns.words != 'exhale expire']
nouns = nouns[nouns.words != 'facetious bantering']
nouns = nouns[nouns.words != 'falsify distort']
nouns = nouns[nouns.words != 'faux fake']
nouns = nouns[nouns.words != 'fils Yemeni_fils']
nouns = nouns[nouns.words != 'flamboyant royal_poinciana']
nouns = nouns[nouns.words != 'francium Fr']
nouns = nouns[nouns.words != 'agronomy scientific_agriculture']
nouns = nouns[nouns.words != 'apt disposed']
nouns = nouns[nouns.words != 'ariette arietta']
nouns = nouns[nouns.words != 'arachnid arachnoid']
nouns = nouns[nouns.words != 'aristocratic aristocratical']
nouns = nouns[nouns.words != 'arbitral arbitrational']
nouns = nouns[nouns.words != 'assist aid']
nouns = nouns[nouns.words != 'automatic automatic_rifle']
nouns = nouns[nouns.words != 'aver allege']
nouns = nouns[nouns.words != 'bandeau brassiere']
nouns = nouns[nouns.words != 'beatify exhilarate']
nouns = nouns[nouns.words != 'bizarre eccentric']
nouns = nouns[nouns.words != 'bourn bourne']
nouns = nouns[nouns.words != 'budge Budge']
nouns = nouns[nouns.words != 'boule boulle']
nouns = nouns[nouns.words != 'briquet briquette']
nouns = nouns[nouns.words != 'bromine Br']
nouns = nouns[nouns.words != 'brumous foggy']
nouns = nouns[nouns.words != 'burnoose burnous']
nouns = nouns[nouns.words != 'bye pass']
nouns = nouns[nouns.words != 'cadaveric cadaverous']
nouns = nouns[nouns.words != 'cajole wheedle']
nouns = nouns[nouns.words != 'cam Cam']
nouns = nouns[nouns.words != 'carrack carack']
nouns = nouns[nouns.words != 'carton cartonful']
nouns = nouns[nouns.words != 'center centre']
nouns = nouns[nouns.words != 'chabazite chabasite']
nouns = nouns[nouns.words != 'champagne bubbly']
nouns = nouns[nouns.words != 'charivari shivaree']
nouns = nouns[nouns.words != 'shivaree chivaree']
nouns = nouns[nouns.words != 'cigarette cigaret']
nouns = nouns[nouns.words != 'cite citation']
nouns = nouns[nouns.words != 'clarify clear_up']
nouns = nouns[nouns.words != 'classify class']
nouns = nouns[nouns.words != 'coy demure']
nouns = nouns[nouns.words != 'colossal prodigious']
nouns = nouns[nouns.words != 'comestible edible']
nouns = nouns[nouns.words != 'complex composite']
nouns = nouns[nouns.words != 'concord Concord']
nouns = nouns[nouns.words != 'compute calculate']
nouns = nouns[nouns.words != 'complaisant obliging']
nouns = nouns[nouns.words != 'coquette flirt']
nouns = nouns[nouns.words != 'console console_table']
nouns = nouns[nouns.words != 'consult confer_with']
nouns = nouns[nouns.words != 'country-dance country_dancing']
nouns = nouns[nouns.words != 'coriander coriander_plant']
nouns = nouns[nouns.words != 'corps army_corps']
nouns = nouns[nouns.words != 'corselet corslet']
nouns = nouns[nouns.words != 'coincide co-occur']
nouns = nouns[nouns.words != 'crochet crocheting']
nouns = nouns[nouns.words != 'cerebral intellectual']
nouns = nouns[nouns.words != 'ceterach Ceterach']
nouns = nouns[nouns.words != 'deluxe gilded']
nouns = nouns[nouns.words != 'demitasse cafe_noir']
nouns = nouns[nouns.words != 'despoil plunder']
nouns = nouns[nouns.words != 'dessert sweet']
nouns = nouns[nouns.words != 'digitigrade digitigrade_mammal']
nouns = nouns[nouns.words != 'diopter dioptre']
nouns = nouns[nouns.words != 'disconcert confuse']
nouns = nouns[nouns.words != 'distrait distracted']
nouns = nouns[nouns.words != 'domino Domino']
nouns = nouns[nouns.words != 'dormition Dormition']
nouns = nouns[nouns.words != 'dot point']
nouns = nouns[nouns.words != 'dugong Dugong_dugon']
nouns = nouns[nouns.words != 'debark disembark']
nouns = nouns[nouns.words != 'dogmatic dogmatical']
nouns = nouns[nouns.words != 'dog domestic_dog']
nouns = nouns[nouns.words != 'decapitate behead']
nouns = nouns[nouns.words != 'decollete low-cut']
nouns = nouns[nouns.words != 'delimit specify']
nouns = nouns[nouns.words != 'defiant noncompliant']
nouns = nouns[nouns.words != 'democratize democratise']
nouns = nouns[nouns.words != 'depute delegate']
nouns = nouns[nouns.words != 'derange unbalance']
nouns = nouns[nouns.words != 'disastrous black']
nouns = nouns[nouns.words != 'deshabille dishabille']
nouns = nouns[nouns.words != 'disorganize disorganise']
nouns = nouns[nouns.words != 'ellipse oval']
nouns = nouns[nouns.words != 'embarrass abash']
nouns = nouns[nouns.words != 'enjambment enjambement']
nouns = nouns[nouns.words != 'inter bury']
nouns = nouns[nouns.words != 'entourage cortege']
nouns = nouns[nouns.words != 'vie compete']
nouns = nouns[nouns.words != 'escargots escargot']
nouns = nouns[nouns.words != 'evince express']
nouns = nouns[nouns.words != 'fibre fiber']
nouns = nouns[nouns.words != 'fiber fibre']
nouns = nouns[nouns.words != 'file data_file']
nouns = nouns[nouns.words != 'focus focusing']
nouns = nouns[nouns.words != 'fusee fusee_drive']
nouns = nouns[nouns.words != 'febrifuge antipyretic']
nouns = nouns[nouns.words != 'federal Federal']
nouns = nouns[nouns.words != 'gaffe faux_pas']
nouns = nouns[nouns.words != 'gibbon Gibbon']
nouns = nouns[nouns.words != 'gauze gauze_bandage']
nouns = nouns[nouns.words != 'gelatine gelatin']
nouns = nouns[nouns.words != 'ha-ha hee-haw']
nouns = nouns[nouns.words != 'hautbois oboe']
nouns = nouns[nouns.words != 'hypnotic soporific']
nouns = nouns[nouns.words != 'idolater idolizer']
nouns = nouns[nouns.words != 'gelatine gelatin']
nouns = nouns[nouns.words != 'hockey field_hockey']
nouns = nouns[nouns.words != 'hyoid hyoid_bone']
nouns = nouns[nouns.words != 'iodine iodin']
nouns = nouns[nouns.words != 'irascible choleric']
nouns = nouns[nouns.words != 'jet jet_plane']
nouns = nouns[nouns.words != 'largess largesse']
nouns = nouns[nouns.words != 'lazaret lazaretto']
nouns = nouns[nouns.words != 'le lupus_erythematosus']
nouns = nouns[nouns.words != 'loir Glis_glis']
nouns = nouns[nouns.words != 'louche shady']
nouns = nouns[nouns.words != 'lucrative moneymaking']
nouns = nouns[nouns.words != 'legume leguminous_plant']
nouns = nouns[nouns.words != 'macabre ghastly']
nouns = nouns[nouns.words != 'maisonette maisonnette']
nouns = nouns[nouns.words != 'mandarin mandarin_orange']
nouns = nouns[nouns.words != 'manganese Mn']
nouns = nouns[nouns.words != 'maquis Maquis']
nouns = nouns[nouns.words != 'match lucifer'] #archaic meaning of lucifer
nouns = nouns[nouns.words != 'milkshake milk_shake']
nouns = nouns[nouns.words != 'milligram mg']
nouns = nouns[nouns.words != 'mollusc mollusk']
nouns = nouns[nouns.words != 'morgue mortuary']
nouns = nouns[nouns.words != 'mot bon_mot']
nouns = nouns[nouns.words != 'mouflon moufflon']
nouns = nouns[nouns.words != 'en nut']
nouns = nouns[nouns.words != 'muscat muskat']
nouns = nouns[nouns.words != 'mystify perplex']
nouns = nouns[nouns.words != 'meter metre']
nouns = nouns[nouns.words != 'metre meter']
nouns = nouns[nouns.words != 'metacenter metacentre']
nouns = nouns[nouns.words != 'metacentre metacenter']
nouns = nouns[nouns.words != 'naive naif']
nouns = nouns[nouns.words != 'nitre potassium_nitrate']
nouns = nouns[nouns.words != 'nitrogen N']
nouns = nouns[nouns.words != 'nocturne notturno']
nouns = nouns[nouns.words != 'offensive offense']
nouns = nouns[nouns.words != 'oleaginous buttery']
nouns = nouns[nouns.words != 'orientalism Orientalism']
nouns = nouns[nouns.words != 'oriole Old_World_oriole']
nouns = nouns[nouns.words != 'orthopedic orthopaedic']
nouns = nouns[nouns.words != 'oxygen O']
nouns = nouns[nouns.words != 'palmier booming']
nouns = nouns[nouns.words != 'panda giant_panda']
nouns = nouns[nouns.words != 'parity para']
nouns = nouns[nouns.words != 'passive passive_voice']
nouns = nouns[nouns.words != 'persecute oppress']
nouns = nouns[nouns.words != 'pickpocket cutpurse']
nouns = nouns[nouns.words != 'piolet ice_ax']
nouns = nouns[nouns.words != 'pipette pipet']
nouns = nouns[nouns.words != 'pipistrelle pipistrel']
nouns = nouns[nouns.words != 'piston Piston']
nouns = nouns[nouns.words != 'plaintive mournful']
nouns = nouns[nouns.words != 'plutonic irruptive']
nouns = nouns[nouns.words != 'polemic polemicist']
nouns = nouns[nouns.words != 'po polonium']
nouns = nouns[nouns.words != 'pragmatic pragmatic_sanction']
nouns = nouns[nouns.words != 'practicable operable']
nouns = nouns[nouns.words != 'prodigious colossal']
nouns = nouns[nouns.words != 'prompt prompting']
nouns = nouns[nouns.words != 'prosaic matter-of-fact']
nouns = nouns[nouns.words != 'prude puritan']
nouns = nouns[nouns.words != 'prehistoric prehistorical']
nouns = nouns[nouns.words != 'ptomaine ptomain']
nouns = nouns[nouns.words != 'pinnace tender']
nouns = nouns[nouns.words != 'periodic periodical']
nouns = nouns[nouns.words != 'reconnoiter scout']
nouns = nouns[nouns.words != 'reconnoitre scout']
nouns = nouns[nouns.words != 'redoubtable formidable']
nouns = nouns[nouns.words != 'remarkable singular']
nouns = nouns[nouns.words != 'regorge vomit']
nouns = nouns[nouns.words != 'remake remaking']
nouns = nouns[nouns.words != 'rally mass_meeting']
nouns = nouns[nouns.words != 'radical group']
nouns = nouns[nouns.words != 'ramify complexify']
nouns = nouns[nouns.words != 'risque blue']
nouns = nouns[nouns.words != 'ruttier rutted']
nouns = nouns[nouns.words != 'rail railing']
nouns = nouns[nouns.words != 'reprimand rebuke']
nouns = nouns[nouns.words != 'resistant immune']
nouns = nouns[nouns.words != 'reveal uncover']
nouns = nouns[nouns.words != 'soar zoom']
nouns = nouns[nouns.words != 'sabre saber']
nouns = nouns[nouns.words != 'sainfoin sanfoin']
nouns = nouns[nouns.words != 'savant initiate']
nouns = nouns[nouns.words != 'settee settle']
nouns = nouns[nouns.words != 'somber drab']
nouns = nouns[nouns.words != 'sumptuous deluxe']
nouns = nouns[nouns.words != 'soporific hypnotic']
nouns = nouns[nouns.words != 'sprint dash']
nouns = nouns[nouns.words != 'stagnant dead']
nouns = nouns[nouns.words != 'tactile haptic']
nouns = nouns[nouns.words != 'talus scree']
nouns = nouns[nouns.words != 'tentative probationary']
nouns = nouns[nouns.words != 'theatre theater']
nouns = nouns[nouns.words != 'timid cautious']
nouns = nouns[nouns.words != 'titer titre']
nouns = nouns[nouns.words != 'titre titer']
nouns = nouns[nouns.words != 'ton short_ton']
nouns = nouns[nouns.words != 'eclectic eclecticist']

In [55]:
nouns['words'] = nouns.words.replace(' ', ', ', regex=True)
nouns['words'] = nouns.words.replace('_', ' ', regex=True)

In [56]:
nouns = nouns[nouns.words != 'eject, chuck_out']
nouns = nouns[nouns.words != 'alert, qui vive']
nouns = nouns[nouns.words != 'attitude, mental attitude']
nouns = nouns[nouns.words != 'bail, bail bond']
nouns = nouns[nouns.words != 'billion, one million million']
nouns = nouns[nouns.words != 'calque, calque formation']
nouns = nouns[nouns.words != 'campaign, political campaign']
nouns = nouns[nouns.words != 'capitalism, capitalist economy']
nouns = nouns[nouns.words != 'cereal, cereal grass']
nouns = nouns[nouns.words != 'clementine, clementine tree']
nouns = nouns[nouns.words != 'crepe, crepe paper']
nouns = nouns[nouns.words != 'croissant, crescent roll']
nouns = nouns[nouns.words != 'deputy, deputy sheriff']
nouns = nouns[nouns.words != 'douche, douche bag']
nouns = nouns[nouns.words != 'fester, suppurating sore']
nouns = nouns[nouns.words != 'guillotine, closure by compartment']
nouns = nouns[nouns.words != 'ideology, political orientation']
nouns = nouns[nouns.words != 'journalism, news media']
nouns = nouns[nouns.words != 'masquerade, masquerade party']
nouns = nouns[nouns.words != 'menu, bill of fare']
nouns = nouns[nouns.words != 'metric, metric function']
nouns = nouns[nouns.words != 'pedal, pedal point']
nouns = nouns[nouns.words != 'positivism, logical positivism']
nouns = nouns[nouns.words != 'prime, prime quantity']
nouns = nouns[nouns.words != 'proprietary, proprietorship']
nouns = nouns[nouns.words != 'psychology, psychological science']
nouns = nouns[nouns.words != 'reconnaissance, reconnaissance mission']
nouns = nouns[nouns.words != 'routine, modus operandi']
nouns = nouns[nouns.words != 'sally, wisecrack']
nouns = nouns[nouns.words != 'sortie, sally']
nouns = nouns[nouns.words != 'terminal, terminus']
nouns = nouns[nouns.words != 'tourniquet, compression bandage']
nouns = nouns[nouns.words != 'vise, bench vise']
nouns = nouns[nouns.words != 'tone, tone of voice']
nouns = nouns[nouns.words != 'syndicate, crime syndicate']
nouns = nouns[nouns.words != 'tangerine, tangerine tree']
nouns = nouns[nouns.words != 'parameter, parametric quantity']
nouns = nouns[nouns.words != 'ounce, troy ounce']
nouns = nouns[nouns.words != 'pretext, stalking-horse']
nouns = nouns[nouns.words != 'lingerie, intimate apparel']
nouns = nouns[nouns.words != 'positivism, logical positivism']
nouns = nouns[nouns.words != 'ballet, concert dance']
nouns = nouns[nouns.words != 'chamois, chamois leather']
nouns = nouns[nouns.words != 'celery, cultivated celery']
nouns = nouns[nouns.words != 'clique, coterie']
nouns = nouns[nouns.words != 'compote, fruit compote']
nouns = nouns[nouns.words != 'contrast, direct contrast']
nouns = nouns[nouns.words != 'date, day of the month']
nouns = nouns[nouns.words != 'defeat, licking']
nouns = nouns[nouns.words != 'diplomacy, diplomatic negotiations']
nouns = nouns[nouns.words != 'promotion, publicity']
nouns = nouns[nouns.words != 'publicity, promotion']
nouns = nouns[nouns.words != 'crayon, wax crayon']
nouns = nouns[nouns.words != 'expertise, expertness']
nouns = nouns[nouns.words != 'hospitality, cordial reception']
nouns = nouns[nouns.words != 'hut, army hut']

In [57]:
nouns.shape
#292

(292, 2)

In [8]:
nouns.to_csv("nouns_fr_final.csv")